<a href="https://colab.research.google.com/github/kavlata/LLM-experiments/blob/main/MCQA-Pretrained-HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
# Solution 1  :https://stackoverflow.com/questions/76397904/generate-the-probabilities-of-all-the-next-possible-word-for-a-given-text
import torch
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

t = GPT2TokenizerFast.from_pretrained("gpt2")
m = GPT2LMHeadModel.from_pretrained("gpt2")

sentence = "I enjoy walking in the"
sentence = " The Rhodesian Bush War was the result , beginning in earnest on 21 December 1972 when ZANLA attacked Altena and Whistlefield Farms near Centenary in the country 's north @-@ east . The Rhodesian Security Forces launched Operation Hurricane in response and"
encoded_text = t(sentence, return_tensors="pt")

#1. step to get the logits of the next token
with torch.inference_mode():
  outputs = m(**encoded_text)

next_token_logits = outputs.logits[0, -1, :]
print(next_token_logits.shape)
print(next_token_logits)

# 2. step to convert the logits to probabilities
next_token_probs = torch.softmax(next_token_logits, -1)

# 3. step to get the top 10
topk_next_tokens= torch.topk(next_token_probs, 10)

#putting it together
print(*[(t.decode(idx), prob) for idx, prob in zip(topk_next_tokens.indices, topk_next_tokens.values)], sep="\n")

torch.Size([50257])
tensor([-138.4181, -138.3555, -140.3119,  ..., -142.0252, -140.3334,
        -139.0948])
(' the', tensor(0.0571))
(' were', tensor(0.0338))
(' destroyed', tensor(0.0305))
(' began', tensor(0.0193))
(' in', tensor(0.0169))
(',', tensor(0.0163))
(' took', tensor(0.0138))
(' attacked', tensor(0.0132))
(' on', tensor(0.0111))
(' launched', tensor(0.0108))


In [ ]:
!pip install sentence-transformers

In [ ]:
#Solution 2
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained SBERT model
# model = SentenceTransformer('bert-base-nli-mean-tokens')
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Example sentences
sentenceA = "destroyed"
word1 = "fought"
word2 = "contended"
word3 = "defended"
word4 = "opposed"

embeddings = model.encode([sentenceA, word1,word2,word3,word4], convert_to_tensor=True)

# Compute cosine similarity between sentence pairs
similarity_score = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
print("Similarity score between sentence1 and sentence2:", similarity_score)
similarity_score = util.pytorch_cos_sim(embeddings[0], embeddings[2]).item()
print("Similarity score between sentence1 and sentence2:", similarity_score)
similarity_score = util.pytorch_cos_sim(embeddings[0], embeddings[3]).item()
print("Similarity score between sentence1 and sentence2:", similarity_score)
similarity_score = util.pytorch_cos_sim(embeddings[0], embeddings[4]).item()
print("Similarity score between sentence1 and sentence2:", similarity_score)
# similarity_score = util.pytorch_cos_sim(embeddings[0], embeddings[5]).item()
# print("Similarity score between sentence1 and sentence2:", similarity_score)
# similarity_score = util.pytorch_cos_sim(embeddings[0], embeddings[6]).item()
# print("Similarity score between sentence1 and sentence2:", similarity_score)

Similarity score between sentence1 and sentence2: 0.3549085855484009
Similarity score between sentence1 and sentence2: 0.3360002934932709
Similarity score between sentence1 and sentence2: 0.4247986972332001
Similarity score between sentence1 and sentence2: 0.34628432989120483


In [ ]:
!pip install pytorch-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.2 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=b27085f7c8d8d5530fa7fb7dc48eab42161ea4e6bfc81eeffe6092485e919d03
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [ ]:
# SOlution 3 : https://stackoverflow.com/questions/62703391/estimate-token-probability-logits-given-a-sentence-without-computing-the-entire
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel
import pandas as pd

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


def score(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    loss = model(tensor_input, labels=tensor_input)
    return -loss[0].item()


candidates = ["watch", "run", "think", "apple", "light"]
sent_template = "I like sitting in my new chair and {} about life"
print({candidate: score(sent_template.format(candidate)) for candidate in candidates})

100%|██████████| 456318/456318 [00:00<00:00, 621482.79B/s]


{'watch': -5.3080339431762695, 'run': -5.4340314865112305, 'think': -4.455368995666504, 'apple': -6.093069076538086, 'light': -5.763839244842529}


In [ ]:
sentenceA = """ The Rhodesian Bush War was the result , beginning in earnest on 21 December 1972 when ZANLA attacked Altena
and Whistlefield Farms near Centenary in the country 's north @-@ east . The Rhodesian Security Forces launched \\
Operation Hurricane in response and {} back strongly , reducing the number of guerrillas active within the country to under 300 by December 1974 . In the period October â€“ November 1974 , they killed more nationalist fighters than in the previous two years combined . However , a South African @-@ brokered ceasefire , which the security forces respected and the insurgents ignored , helped the nationalists to win back ground . By 1977 there were 2 @,@ 500 guerrillas operating in Rhodesia , with several times that number in training abroad ."""
word1 = "fought"
word2 = "contended"
word3 = "defended"
word4 = "opposed"

candidates = [word1,word2,word3,word4]
print({candidate: score(sentenceA.format(candidate)) for candidate in candidates})

{'fought': -4.5446977615356445, 'contended': -4.630280017852783, 'defended': -4.623897075653076, 'opposed': -4.6275177001953125}


In [ ]:
#Solution 4: https://stackoverflow.com/questions/61787853/how-to-get-the-probability-of-a-particular-tokenword-in-a-sentence-given-the-c
from transformers import AutoTokenizer, BertForMaskedLM

tok = AutoTokenizer.from_pretrained("bert-base-cased")
bert = BertForMaskedLM.from_pretrained("bert-base-cased")

sentenceA = f""" The Rhodesian Bush War was the result , beginning in earnest on 21 December 1972 when ZANLA attacked Altena
and Whistlefield Farms near Centenary in the country 's north @-@ east . The Rhodesian Security Forces launched \\
Operation Hurricane in response and {tok.mask_token} back strongly , reducing the number of guerrillas active within the country to under 300 by December 1974 . In the period October â€“ November 1974 , they killed more nationalist fighters than in the previous two years combined . However , a South African @-@ brokered ceasefire , which the security forces respected and the insurgents ignored , helped the nationalists to win back ground . By 1977 there were 2 @,@ 500 guerrillas operating in Rhodesia , with several times that number in training abroad ."""

# sentencesA = f"The {tok.mask_token} were the best rock band ever."
input_idx = tok.encode(sentenceA)
logits = bert(torch.tensor([input_idx]))[0]
prediction = logits[0].argmax(dim=1)
print(tok.convert_ids_to_tokens(prediction[2].numpy().tolist()))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Rhodesia


In [ ]:
#Solution 5 : https://huggingface.co/docs/transformers/tasks/multiple_choice Then fine tune
from transformers import AutoTokenizer, RobertaForMultipleChoice
import torch

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForMultipleChoice.from_pretrained("roberta-base")

prompt = "The Rhodesian Bush War was the result , beginning in earnest on 21 December 1972 when ZANLA attacked Altena and Whistlefield Farms near Centenary in the country 's north @-@ east . The Rhodesian Security Forces launched Operation Hurricane in response and [BLANK] back strongly , reducing the number of guerrillas active within the country to under 300 by December 1974 . In the period October â€“ November 1974 , they killed more nationalist fighters than in the previous two years combined . However , a South African @-@ brokered ceasefire , which the security forces respected and the insurgents ignored , helped the nationalists to win back ground . By 1977 there were 2 @,@ 500 guerrillas operating in Rhodesia , with several times that number in training abroad ."
choice0 = "fought"
choice1 = "contended"
choice2 = "defended"
choice3 = "opposed"

labels = torch.tensor(0).unsqueeze(0)  # choice0 is correct (according to Wikipedia ;)), batch size 1

encoding = tokenizer([prompt, prompt,prompt,prompt], [choice0, choice1,choice2,choice3], return_tensors="pt", padding=True)
outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()}, labels=labels)  # batch size is 1

# the linear classifier still needs to be trained
loss = outputs.loss
logits = outputs.logits

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultipleChoice: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream t

In [ ]:
answer_choices = [choice0,choice1, choice2,choice3]
predicted_answer_index = torch.argmax(logits, dim=1).item()
predicted_answer = answer_choices[predicted_answer_index]

print("Predicted Answer:", predicted_answer)

Predicted Answer: defended


In [ ]:
from transformers import AutoTokenizer, LongformerForMultipleChoice
import torch

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")
model = LongformerForMultipleChoice.from_pretrained("allenai/longformer-base-4096")

prompt = "The Rhodesian Bush War was the result , beginning in earnest on 21 December 1972 when ZANLA attacked Altena and Whistlefield Farms near Centenary in the country 's north @-@ east . The Rhodesian Security Forces launched Operation Hurricane in response and [BLANK] back strongly , reducing the number of guerrillas active within the country to under 300 by December 1974 . In the period October â€“ November 1974 , they killed more nationalist fighters than in the previous two years combined . However , a South African @-@ brokered ceasefire , which the security forces respected and the insurgents ignored , helped the nationalists to win back ground . By 1977 there were 2 @,@ 500 guerrillas operating in Rhodesia , with several times that number in training abroad ."
choice0 = "fought"
choice1 = "contended"
choice2 = "defended"
choice3 = "opposed"
labels = torch.tensor(0).unsqueeze(0)  # choice0 is correct (according to Wikipedia ;)), batch size 1

encoding = tokenizer([prompt, prompt,prompt,prompt], [choice0, choice1,choice2,choice3], return_tensors="pt", padding=True)
outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()}, labels=labels)  # batch size is 1

# the linear classifier still needs to be trained
loss = outputs.loss
logits = outputs.logits

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForMultipleChoice: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing LongformerForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForMultipleChoice were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a dow

In [ ]:
answer_choices = [choice0,choice1, choice2,choice3]
predicted_answer_index = torch.argmax(logits, dim=1).item()
predicted_answer = answer_choices[predicted_answer_index]

print("Predicted Answer:", predicted_answer)

Predicted Answer: defended
